In [4]:
from hapi import *
import numpy as np
import matplotlib as plt
import pickle

HAPI version: 1.1.0.9.6
To get the most up-to-date version please check http://hitran.org/hapi
ATTENTION: Python versions of partition sums from TIPS-2017 are now available in HAPI code

           It is free to use HAPI. If you use HAPI in your research or software development,
           please cite it using the following reference:
           R.V. Kochanov, I.E. Gordon, L.S. Rothman, P. Wcislo, C. Hill, J.S. Wilzewski,
           HITRAN Application Programming Interface (HAPI): A comprehensive approach
           to working with spectroscopic data, J. Quant. Spectrosc. Radiat. Transfer 177, 15-30 (2016)
           DOI: 10.1016/j.jqsrt.2016.03.005


In [5]:
db_begin('data')

AllParamGroups = ['160-char','Standard','LineMixing',
                  'Voigt','SDVoigt','Galatry']

wavMin,wavMax = 3400,4100

Using data

CH4
                     Lines parsed: 20131
O3
                     Lines parsed: 25367
H2
                     Lines parsed: 121
H2O
                     Lines parsed: 7535
CO2
                     Lines parsed: 29269
O2
                     Lines parsed: 489


In [6]:
fetch('H2O',1,1,wavMin,wavMax)
fetch('CO2',2,1,wavMin,wavMax)


Data is fetched from http://hitran.org

BEGIN DOWNLOAD: H2O
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
  65536 bytes written to data/H2O.data
Header written to data/H2O.header
END DOWNLOAD
                     Lines parsed: 7535
PROCESSED

Data is fetched from http://hitran.org

BEGIN DOWNLOAD: CO2
  65536 bytes written to data/CO2.data
 

In [7]:
fetch('O3',3,1,wavMin,wavMax)
fetch('CH4',6,1,wavMin,wavMax)


Data is fetched from http://hitran.org

BEGIN DOWNLOAD: O3
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to data/O3.data
  65536 bytes written to dat

In [8]:
#fetch('N2',22,1,wavMin,wavMax)
fetch('H2',45,1,wavMin,wavMax)


Data is fetched from http://hitran.org

BEGIN DOWNLOAD: H2
  65536 bytes written to data/H2.data
Header written to data/H2.header
END DOWNLOAD
                     Lines parsed: 121
PROCESSED


In [9]:
molDict = {1:'H2O',2:'CO2',3:'O3',
            6:'CH4',22:'N2',45:'H2'}

In [10]:
def crossSection(molSet,p=1.0,T=296.0,profile='Voigt',gamma='air'):
    # wavRange = wavenumber tuple min, max
    # molSet = list of tables
    # envir = pressure temp dict
    # profile = coeff calculator
    # gamma = self, air, etc
    if profile in ['HT','Voigt','Lorentz','Doppler']:
        func = eval('absorptionCoefficient_'+profile)
    else:
        raise ValueError('Line shape must be HT, Voigt, Lorentz, or Doppler.')
    if gamma in ['air','self']:
        gL = 'gamma_'+gamma
    envir = {'p':p,'T':T}
    nu,coef = func(SourceTables=molSet,Environment=envir,GammaL=gL)
    return (nu,coef)

In [11]:
%timeit crossSection([molDict[1]],p=1.0,T=296.0,profile='Voigt',gamma='air')

{'air': 1.0}
{'air': 1.0}
{'air': 1.0}
{'air': 1.0}
1 loop, best of 3: 12.5 s per loop


In [12]:
def databaseGen(tRange,tStep,pRange,pStep,compList,pickleSave=True):
    # tRange, pRange = temp and pressure tuples with min, max
    # tRange, pStep = temp and presure step size
    # compList = list of molecule numbers
    tArange = np.arange(tRange[0],tRange[1]+tStep,tStep)
    pArange = np.arange(pRange[0],pRange[1]+pStep,pStep)
    wavNums = len(crossSection([molDict[compList[1]]])[0])
    db = np.empty((len(tArange),len(pArange),len(compList),wavNums))
    for t in range(len(db)):
        tVal = tArange[t]
        for p in range(len(db[t])):
            pVal = pArange[p]
            for c in range(len(db[t][p])):
                cVal = compList[c]
                mol = molDict[cVal]
                nu,coef = crossSection([mol],p=pVal,T=tVal)
                db[t][p][c] = coef
    if pickleSave:
        filename = 'db'+str(tRange[0])+str(tRange[1])+str(tStep)+str(pRange[0])+str(pRange[1])+str(pStep)
        outfile = open(filename,'wb')
        pickle.dump(db,outfile)
        outfile.close()
    return db

In [ ]:
tRange=(100,300)
tStep=100
pRange=(1,3)
pStep=1
compList=[1]
res = databaseGen(tRange,tStep,pRange,pStep,compList)

In [ ]:
nu = crossSection([molDict[1]],p=1.0,T=296.0,profile='Voigt',gamma='air')
res.shape